In [1]:
import ccxt
import pandas as pd
from pymongo import MongoClient

In [2]:
mongo_client = MongoClient('localhost', 27017)
db = mongo_client["ohlc"]
market = db["binance"]

In [3]:
def get_file_contents(filename):
    try:
        with open(filename, 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [4]:
binance = ccxt.binance()
binance.apiKey = get_file_contents('/home/nam/api_key/binance/pub')
binance.secret = get_file_contents('/home/nam/api_key/binance/private')

In [5]:
# upon instantiation
binance = ccxt.binance({
    'apiKey': binance.apiKey,
    'secret': binance.secret,
})

exchange = ccxt.binance({
    # 'apiKey': binance.apiKey,
    # 'secret': binance.secret,
    'timeout': 30000,
    'rateLimit': 2000,
    'enableRateLimit': True
})

# symbols = ['BTC/USDT']

In [6]:
import time
msec = 1000
minute = 60 * msec
hour = 60 * minute
hold = 30

from_datetime = '2019-03-18 00:00:00'
from_timestamp = exchange.parse8601(from_datetime)
to_datetime = '2019-05-20 00:00:00'
to_timestamp = exchange.parse8601(to_datetime)

In [7]:
data = []
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']


collection = db[]
while from_timestamp < to_timestamp:
    try:
        # print(exchange.milliseconds(), 'Fetching candles starting from', exchange.iso8601(from_timestamp))
        ohlcvs = exchange.fetch_ohlcv('BTC/USDT', '1h', from_timestamp)
        # df_current = pd.DataFrame(list(ohlcvs), columns = header)
        df_current = pd.DataFrame(ohlcvs, columns = header)
        # convert df to list of dict
        lst_dict = df_current.T.to_dict().values()
        collection.insert_many(lst_dict)

        print(exchange.milliseconds(), 'Fetched', len(ohlcvs), 'candles')
        if len(ohlcvs) > 0:
            first = ohlcvs[0][0]
            last = ohlcvs[-1][0]
            print('First candle epoch', first, exchange.iso8601(first))

            # from_timestamp += len(ohlcvs) * minute * 5  # very bad
            from_timestamp = ohlcvs[-1][0] + minute * 5  # good
            # from_timestamp = ohlcvs[-1][0] 
            data += ohlcvs

            # v = ohlcvs[0][0]/ 1000
            # !date --date @{v} +"%Y-%m-%d %H:%M"
            print('Last candle epoch', last, exchange.iso8601(last))

    except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:
        print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
        time.sleep(hold)

1564223276841 Fetched 500 candles
First candle epoch 1552867200000 2019-03-18T00:00:00.000Z
Last candle epoch 1554663600000 2019-04-07T19:00:00.000Z
1564223278839 Fetched 500 candles
First candle epoch 1554667200000 2019-04-07T20:00:00.000Z
Last candle epoch 1556463600000 2019-04-28T15:00:00.000Z
1564223280842 Fetched 500 candles
First candle epoch 1556467200000 2019-04-28T16:00:00.000Z
Last candle epoch 1558299600000 2019-05-19T21:00:00.000Z
1564223282839 Fetched 500 candles
First candle epoch 1558303200000 2019-05-19T22:00:00.000Z
Last candle epoch 1560099600000 2019-06-09T17:00:00.000Z


In [14]:
df_current.Timestamp.head()

0    1558303200000
1    1558306800000
2    1558310400000
3    1558314000000
4    1558317600000
Name: Timestamp, dtype: int64

In [20]:
df_current["symbols"] = "BNB/BTC"
df_current["timescale"] = "1h"
df_current["market"] = "binance"
df_current

,Timestamp,Open,High,Low,Close,Volume,symbols,timediff,market
0,1558303200000,8058.19,8202.44,7990.00,8188.72,4354.550893,BNB/BTC,1h,binance
1,1558306800000,8188.72,8275.09,8110.02,8148.48,4380.887254,BNB/BTC,1h,binance
2,1558310400000,8147.94,8156.03,7780.00,7907.18,7200.384192,BNB/BTC,1h,binance
3,1558314000000,7905.91,7986.64,7841.86,7965.15,2491.291779,BNB/BTC,1h,binance
4,1558317600000,7965.08,7966.78,7830.00,7889.32,1755.837013,BNB/BTC,1h,binance
5,1558321200000,7889.33,8067.30,7885.85,8030.04,2082.664391,BNB/BTC,1h,binance
6,1558324800000,8030.04,8109.81,7984.72,7996.00,2177.737479,BNB/BTC,1h,binance
7,1558328400000,7996.00,8005.20,7920.00,7985.82,1388.324550,BNB/BTC,1h,binance
8,1558332000000,7985.76,7990.01,7850.00,7870.00,1960.055352,BNB/BTC,1h,binance
9,1558335600000,7870.00,7966.00,7829.79,7871.41,2811.888708,BNB/BTC,1h,binance


In [10]:
import json
JSON_FILE = "../json/binance-1hours"
cursor = collection.find({})
file = open(JSON_FILE, "w")
file.write('[')
for document in cursor:
    file.write(json.dumps(document))
    file.write(',')
#     print(json.dumps(document))
#     print(document)
file.write(']')

TypeError: Object of type 'ObjectId' is not JSON serializable

In [12]:
# !mongoexport --db ohlc-min --collection binance --out $JSON_FILE

2019-07-27T17:29:12.763+0700	connected to: localhost
2019-07-27T17:29:12.825+0700	exported 2500 records
